# Lecture 10

## Function approximation by the finite element method

We have seen that a function $u(x)$ can be approximated as

$$
u(x) = \sum_{j=0}^N \hat{u}_j \psi_j(x), \quad x \in \Omega.
$$

And we have seen how the least squares, Galerkin and collocation methods can be used to find the unknown $\boldsymbol{\hat{u}} = \{\hat{u}_j\}_{j=0}^N$. Up until now we have used global basis functions $\psi_j(x)$ defined on the entire domain $\Omega$. For example, the Chebyshev or Legendre polynomials are all functions that are changing within the entire domain $\Omega = [-1, 1]$

The global aspect of the basis functions is an advantage when it comes to both accuracy and efficiency. However, we are not always interested in solving equations on a simple line interval, or a rectangle for two dimensions. Normally we are more interested in domains that contain some physical obstructions

```{figure} dolfin_mesh.png
:name: dolfin mesh
:width: 300
:align: left
```

For such domains it is not possible to use basis functions that are defined everywhere. It is also very difficult, if not impossible, to use finite difference methods. Just imagine, how would you implement a Laplacian, like

$$
\frac{u_{i+1,j}-2u_{i,j}+u_{i-1,j}}{\Delta x^2} + \frac{u_{i,j+1}-2u_{i,j}+u_{i,j-1}}{\Delta y^2} 
$$

on a 2D dolfin mesh?

The finite element methods, on the other hand, are designed to work on such triangulated and unstructured meshes in complex domains. But in order to present the method it makes sense to first stick to simple one-dimensional domains also here.

The finite element method (FEM) starts by splitting the domain $\Omega$ into $N_e$ smaller, non-overlapping, subdomains $\Omega^{(e)}$, such that

$$
\Omega = \bigcup_{e\in \mathcal{I}_{N_e}} \Omega^{(e)}.
$$

For example, the domain $\Omega = [0, 5]$ can be split up into 5 smaller subdomains $\Omega^{(e)} = [e, e+1], e \in (0, 1, 2, 3, 4)$ using 6 uniformly distributed nodes:

```{figure} fe_mesh1D_P1.png
---
name: fem-1d-mesh-P1
width: 500
align: left
---
Finite element mesh with 5 elements and 6 nodes.
```
These smaller subdomains are now referred to as **elements**.
Note that an element here is located in between two blue vertical bars. 

The mesh seen in {numref}`fem-1d-mesh-P1` is the simplest possible FEM mesh, where each element contains 2 *nodes*. These elements can at best make use of linear piecewise polynomials since there are only 2 nodes in each element.

A slightly more complicated mesh is shown in {numref}`fem-1d-mesh-P2`, which is non-uniform and contains only 2 elements and 5 nodes. Each of the elements contains 3 nodes, and as such these elements can make use of second order polynomials.

```{figure} fe_mesh1D_P2.png
---
name: fem-1d-mesh-P2
width: 500
align: left
---
Finite element mesh with 2 elements and 5 nodes.
```

The FEM makes use of *local* basis functions that are only non-zero on some of the elements. Furthermore, the basis functions that we will make use of in this class are piecewise polynomials. In the simplest possible form, this means that the basis functions are piecewise linear (or even constant) over an element. For example, for the mesh in {numref}`fem-1d-mesh-P1` we plot two of the 6 basis functions below:

```{figure} fe_mesh1D_phi_2_3.png
---
name: fem-1d-mesh-phi
width: 500
align: left
---
Finite element mesh with 5 elements, 6 nodes and two of the 6 basis functions.
```

### Finite element basis functions

Other than the fact that the basis functions are piecewise polynomials with only local support, there is nothing new from the previous chapters. It is only much more complicated to work with piecewise polynomials than continuous and it requires much more effort to implement

The finite element method is a variational method. As such we can define the problem of approximating functions exactly the same way as we did for the global approach. Assume that we want to use the finite element method to find an approximation to $u(x)$ in $\Omega=[0, 5]$. If we use the mesh in {numref}`fem-1d-mesh-P1` we get a funciton space $V_N = \text{span}\{\psi_j\}_{j=0}^5$, where $\psi_j(x)$ are the 6 piecewise linear functions that are 1 on node $j$, zero on all the other nodes and linear in between. We then attempt to find $u_N \in V_N$ such that

$$
(u-u_N, v) = 0 \quad \forall \, v \in V_N.
$$(eq-galerkin-fem)

It is also possible to use the least squares method, but this is very rare. So for the finite element method we will focus on the Galerkin method with piecewise polynomial basis functions.

```{note}
The Galerkin formulation is the same whether you use a global approach with Legendre polynomials or a local FEM with piecewise polynomials. The difference lies all in the function spaces and the choice of basis. 
```

We will reuse even more of the theory from the global approximation, because we will in this class only make use of Lagrange polynomials as basis functions! But not global Lagrange polynomials, only local. Local to an element. Each element in {numref}`fem-1d-mesh-P1` contains $N_e = 2$ nodes. These nodes will locally be denoted as $(x^0, x^1)$ no matter which element we are in, and the Lagrange formula is then

$$
\ell_j(x) = \prod_{\substack{0 \le m < 2 \\ m \ne j}} \frac{x-x^m}{x^j-x^m}, \quad j \in (0, 1).
$$

There are exactly two linear Lagrange polynomials on each element, and these are locally computed as

$$
\ell_0(x) = \frac{x-x^1}{x^0-x^1} \quad \text{and} \quad \ell_1(x) = \frac{x-x^0}{x^1-x^0}.
$$

We can plot all these basis functions with one color for each basis function $\psi_j(x)$

In [ ]:
import numpy as np
import sympy as sp 
import matplotlib.pyplot as plt 
x = sp.Symbol('x')

color = iter(plt.cm.rainbow(np.linspace(0, 1, 7)))
c = next(color)
plt.figure(figsize=(6, 3))
for j in range(5):
    xj = np.array([j, j+1])
    plt.plot(xj,  j+1-xj, c=c)
    c = next(color)
    plt.plot(xj,  xj-j, c=c)

So all the internal basis functions, i.e., $\{\psi_j(x)\}_{j=1}^{4}$, are nonzero in two elements each and zero elsewhere. The two boundary basis functions $\psi_0(x)$ and $\psi_5(x)$ are only nonzero on one element each. All basis functions are such that

$$
\psi_j(x_i) = \delta_{ij} = \begin{cases}
1 \quad i=j \\
0 \quad i\ne j
\end{cases}
$$

since we are using Lagrange polynomials. This applies not only for uniform meshes, but for any collection of mesh points. Assume that the mesh points use Chebyshev points instead. This is no problem for the elements of the Lagrange polynomials.

In [ ]:
xj = np.cos(np.arange(6)*np.pi/5)[::-1]
color = iter(plt.cm.rainbow(np.linspace(0, 1, 7)))
c = next(color)
plt.figure(figsize=(6, 3))
for j in range(5):
    x = np.array([xj[j], xj[j+1]])
    plt.plot(x,  (x-x[1])/(x[0]-x[1]), c=c)
    c = next(color)
    plt.plot(x,  (x-x[0])/(x[1]-x[0]), c=c)

The flexibility when it comes to mesh is one of the major advantages of the finite element method.

For any mesh we get the piecewise linear Lagrange basis functions

$$
\psi_j(x) = \begin{cases}
\frac{x-x_{j-1}}{x_{j}-x_{j-1}} \quad x \in [x_{j-1}, x_{j}],\\
\frac{x-x_{j+1}}{x_{j}-x_{j+1}} \quad x \in [x_{j}, x_{j+1}]. \\
\end{cases}
$$

We have the function space $V_N = \text{span}\{\psi_j\}_{j=0}^N$ and with the Galerkin method {eq}`eq-galerkin-fem` we approximate any function $u(x)$ by looking for $u_N \in V_N$ such that

$$
(u-\sum_{j=0}^N \hat{u}_j \psi_j, \psi_i) = 0 \quad \forall \, i \in (0, 1, \ldots, N).
$$

In other words, we need to solve the follow linear algebra problem

$$
\sum_{j=0}^N(\psi_j, \psi_i) \hat{u}_j = (u, \psi_i), \quad \forall\,  i \in (0, 1, \ldots, N).
$$

Unfortunately, the finite element mass matrix $A = (a_{ij})_{i,j=0}^N$, with

$$
a_{ij} = (\psi_j, \psi_i) = \int_{\Omega} \psi_j \psi_i d\Omega,
$$

is not diagonal, because the basis functions are not orthogonal. However, since the basis functions are local the matrix will still be highly sparse. 

Since the basis functions are piecewise polynomials and only some of the basis fucntions are nonzero on each element, we usually assemble the matrix elementwise as

$$
a_{ij} = \sum_{e=0}^{N_e} \int_{\Omega^{(e)}} \psi_j \psi_i d\Omega 
$$

Here we will use the element matrix $a_{ij}^{(e)} = \int_{\Omega^{(e)}} \psi_j \psi_i d\Omega$. Note that this matrix is still of shape $(N+1) \times (N+1)$, but for any given element it will only consist of a few nonzero items. We have

$$
A = \sum_{e=0}^{N_e-1} A^{(e)}.
$$

Since $A^{(e)}$ is so sparse (it is only nonzero for indices $i, j$ in the element $e$) it makes sense to reduce its size. If there are $d+1$ nonzero basis functions on each element, then there can be at most $(d+1)^2$ combinations of $i$ and $j$ in $(\psi_i, \psi_j)$ and the element matrix $\tilde{A}^{(e)}$ can as such be represented as a dense $(d+1)\times (d+1)$ matrix

$$
\tilde{A}^{(e)} = (a_{rs}^{(e)})_{r,s=0}^{d} \in \mathbb{R}^{(d+1)\times (d+1)}
$$

where

$$
a_{rs}^{(e)} = \int_{\Omega^{(e)}} \psi_{q(e,r)} \psi_{q(e,s)} d\Omega
$$

and $q(e,r)$ is a mapping function that maps a local index $r$ on the global element $e$ to a global index. For an element with $d+1$ nodes we get

$$
q(e, r) = de+r.
$$

```{note}
The matrix $\tilde{A}^{(e)}$ contains the same nonzero items as $A^{(e)}$, but $\tilde{A}^{(e)}\in \mathbb{R}^{(d+1) \times (d+1)}$ is dense, whereas $A^{(e)} \in \mathbb{R}^{(N+1)\times (N+1)}$ is highly sparse.
```

In order to use the local (and small) element matrix $\tilde{A}^{(e)}$ we assemble for each element into the global matrix $A$ as

$$
a_{q(e,r),q(e, s)} \mathop{+}= a^{(e)}_{r,s}, \quad (r, s) \in \mathcal{I}_d^2.
$$

The whole global matrix $A$ can as such be assembled by looping over all elements, like

$$
A = \sum_{e=0}^{N_e-1} \sum_{r=0}^{d}\sum_{s=0}^{d} a^{(e)}_{r,s}.
$$

This process can be illustrated as shown below, where 4 small element matrices $\tilde{A}^{(e)}$ of shape $2 \times 2$ are inserted into the global matrix $A$ of shape $5 \times 5$. For this linear case $d=1$. Note that there is overlap between the 4 element matrices in the global $A$, because all the internal (linear) basis functions are nonzero in 2 elements each.

```{image} movie.gif
:alt: assemble
:width: 400px
:align: center
```

Lets use these basis functions for the Galerkin method in order to approximate $u(x) = 10(x-1)^2-1, x \in [1, 2]$. 

Lets create a uniform mesh and assemble the mass matrix. We assemble element by element and we know that on each element there are only two nonzero basis functions

$$
\ell_0(x) = \frac{x-x^1}{x^0-x^1} \quad \text{and} \quad \ell_1(x) = \frac{x-x^0}{x^1-x^0}.
$$

Since the mesh is uniform, we have $h=\Delta x$ and

$$
\ell_0(x) = \frac{x^1-x}{h} \quad \text{and} \quad \ell_1(x) = \frac{x-x^0}{h}.
$$

For any element

$$
x^{e}_0 = eh \quad x^{e}_1 = (e+1)h
$$



On element $i$ we have that $\psi_i = \ell_0(x)$ and $\psi_{i+1} = \ell_1(x)$. On each element we get the nonzero terms of the mass matrix

$$
\tilde{A}^{(e)} = \begin{bmatrix} 
\int_{\Omega^{(e)}} \psi_{q(e, 0)} \psi_{q(e, 0)} dx &
\int_{\Omega^{(e)}} \psi_{q(e, 0)} \psi_{q(e, 1)} dx \\
\int_{\Omega^{(e)}} \psi_{q(e, 1)} \psi_{q(e, 0)} dx &
\int_{\Omega^{(e)}} \psi_{q(e, 1)} \psi_{q(e, 1)} dx
\end{bmatrix}
$$

$$
\tilde{A}^{(e)} = \begin{bmatrix} 
\int_{\Omega^{(e)}} \psi_{q(e, 0)} \psi_{q(e, 0)} dx &
\int_{\Omega^{(e)}} \psi_{q(e, 0)} \psi_{q(e, 1)} dx \\
\int_{\Omega^{(e)}} \psi_{q(e, 1)} \psi_{q(e, 0)} dx &
\int_{\Omega^{(e)}} \psi_{q(e, 1)} \psi_{q(e, 1)} dx
\end{bmatrix}
$$

Note the addition, which is there because each basis function is nonzero on two elements. Also note that the last two integrals are equal due to symmetry.

We can compute this as below

In [ ]:
import sympy as sp
x = sp.Symbol('x')
N = 4
xj = np.linspace(1, 2, N+1)
l0 = lambda j: (x-xj[j+1])/(xj[j]-xj[j+1])
l1 = lambda j: (x-xj[j])/(xj[j+1]-xj[j])
A = np.zeros((N+1, N+1))
for i in range(N):
    A[i, i] += sp.integrate(l0(i)**2, (x, xj[i], xj[i+1]))
    A[i+1, i+1] += sp.integrate(l1(i)**2, (x, xj[i], xj[i+1]))
    aij = sp.integrate(l0(i)*l1(i), (x, xj[i], xj[i+1]))
    A[i, i+1] += aij
    A[i+1, i] += aij

In [ ]:
print(A)

We can plot the sparsity pattern of the matrix `A` using [plt.spy](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.spy.html). This function plots a square for all the nonzero items of `A`, and nothing where it is zero.

In [ ]:
plt.figure(figsize=(3, 3))
plt.spy(A, markersize=5);

We also need to assemble $b_i = (u, \psi_i)$ for $i=(0, 1, \ldots, N)$:

In [ ]:
b = np.zeros(N+1)
u = 10*(x-1)**2-1
for i in range(N):
    b[i] += sp.integrate(l0(i)*u, (x, xj[i], xj[i+1]))
    b[i+1] += sp.integrate(l1(i)*u, (x, xj[i], xj[i+1]))

Now solve the problem and plot the solution

In [ ]:
uh = np.linalg.inv(A) @ b
yj = np.linspace(1, 2, 1000)
plt.figure(figsize=(5, 3))
plt.plot(xj, uh, 'b', yj, 10*(yj-1)**2-1, 'r--')
plt.legend(['FEM', 'Exact']);

Not bad, but clearly not exact since the FEM solution is piecewise linear between the 5 mesh points.

The next level of complexity for FEM is to increase the order of the approximating polynomials. To this end we need to create elements $\Omega^{(e)}$ that contains more than just 2 nodes each, as shown in {numref}`fem-1d-mesh-P2`.

Lets try with a uniform mesh and 3 nodes in each element. We now use on each element the local mesh points $(x^0, x^1, x^2)$ and get the three Lagrange polynomials

$$
\begin{align*}
\ell_0(x) &= \frac{x-x^1}{x^0-x^1}\frac{x-x^2}{x^0-x^2}, \\
\ell_1(x) &= \frac{x-x^0}{x^1-x^0}\frac{x-x^2}{x^1-x^2}, \\
\ell_2(x) &= \frac{x-x^0}{x^2-x^0}\frac{x-x^1}{x^2-x^1}.
\end{align*}
$$

We can plot these three basis functions on one element as shown below

In [ ]:
from lagrange import Lagrangebasis
fig = plt.figure(figsize=(3, 2))
xj = np.array([0, 0.5, 1])
yj = np.linspace(0, 1, 100)
l0, l1, l2 = Lagrangebasis(xj)
plt.plot(yj, sp.lambdify(x, l0)(yj), 'k', 
         yj, sp.lambdify(x, l1)(yj), 'b',
         yj, sp.lambdify(x, l2)(yj), 'r')
ax = fig.gca()
ax.text(1.05, 0.75, "$\ell_2(x)$", horizontalalignment='center')
ax.text(-0.05, 0.75, "$\ell_0(x)$", horizontalalignment='center')
ax.text(0.5, 0.85, "$\ell_1(x)$", horizontalalignment='center')
ax.text(0, -0.2, "$x^0$", horizontalalignment='center')
ax.text(0.5, -0.2, "$x^1$", horizontalalignment='center')
ax.text(1, -0.2, "$x^2$", horizontalalignment='center')
ax.axis('off');

Easy enough, but what about the global basis numbers? What is $\psi_j(x)$ for node $j$? Since there are 3 nodes in each element, we just need to know which one of these three nodes $j$ corresponds to. And if $\mod(j, 2) = 0$, then $j$ corresponds to the leftmost node, whereas $\mod(j, 2) = 1$ and 2 corresponds to the nodes in the middle and far right. We get the global basis functions that are valid for any $j$

$$
\psi_j(x) = \begin{cases}
\ell_0(x) = \frac{x-x_{j+1}}{x_{j}-x_{j+1}}\frac{x-x_{j+2}}{x_{j}-x_{j+2}}, \, \text{if} \, \mod(j, 2) = 0, \\
\ell_1(x) = \frac{x-x_{j}}{x_{j+1}-x_j}\frac{x-x_{j+2}}{x_{j+1}-x_{j+2}}, \, \text{if} \, \mod (j, 2) = 1, \\
\ell_2(x) = \frac{x-x_j}{x_{j+2}-x_j}\frac{x-x_{j+1}}{x_{j+2}-x_{j+1}} \, \text{if} \, \mod(j, 2) = 2.
\end{cases}
$$

These basis functions are also valid for a non-uniform mesh. For a uniform mesh simplifications are possible, but we skip over this for now.

For the Galerkin method we need to solve

$$
\sum_{j=0}^N (\psi_j, \psi_i) \hat{u}_j = (u, \psi_i), \, \forall \, i \in (0, 1, \ldots, N).
$$

The mass matrix $(\psi_j, \psi_i)$ will now be a little more complicated than for linear polynomials, because there are three nonzero basis functions on each element. To simplify, for the leftmost node, where $\mod(j, 2) = 0$, we need to compute the nonzero terms

$$
(\psi_j, \psi_j), \, (\psi_{j+1}, \psi_j), \, \text{and} \, (\psi_{j+2}, \psi_j).
$$

For the central nodes, where $\mod(j, 2) = 1$, we need to compute

$$
(\psi_{j-1}, \psi_j), \, (\psi_{j}, \psi_j), \, \text{and} \, (\psi_{j+1}, \psi_j),
$$

and finally for the rightmost node we need to compute

$$
(\psi_j, \psi_j), \, (\psi_{j-1}, \psi_j), \, \text{and} \, (\psi_{j-2}, \psi_j).
$$

For implementation we loop over all elements, and then on each element we compute the nine contributions outlined above.

In [ ]:
from scipy.integrate import quad

def assemble_fem2(u, xj):
    N = len(xj)-1
    Ne = N//2
    A = np.zeros((N+1, N+1))
    b = np.zeros(N+1)
    for elem in range(Ne):
        j0 = 2*elem
        ll = Lagrangebasis(xj[j0:(j0+3)])
        # Leftmost nodes
        A[j0, j0] += quad(sp.lambdify(x, ll[0]*ll[0]), xj[j0], xj[j0+2])[0]
        A[j0, j0+1] += quad(sp.lambdify(x, ll[0]*ll[1]), xj[j0], xj[j0+2])[0]
        A[j0, j0+2] += quad(sp.lambdify(x, ll[0]*ll[2]), xj[j0], xj[j0+2])[0]
        # Central nodes
        A[j0+1, j0] += quad(sp.lambdify(x, ll[1]*ll[0]), xj[j0], xj[j0+2])[0]
        A[j0+1, j0+1] += quad(sp.lambdify(x, ll[1]*ll[1]), xj[j0], xj[j0+2])[0]
        A[j0+1, j0+2] += quad(sp.lambdify(x, ll[1]*ll[2]), xj[j0], xj[j0+2])[0]
        # Rightmost nodes
        A[j0+2, j0] += quad(sp.lambdify(x, ll[2]*ll[0]), xj[j0], xj[j0+2])[0]
        A[j0+2, j0+1] += quad(sp.lambdify(x, ll[2]*ll[1]), xj[j0], xj[j0+2])[0]
        A[j0+2, j0+2] += quad(sp.lambdify(x, ll[2]*ll[2]), xj[j0], xj[j0+2])[0]
        b[j0] += quad(sp.lambdify(x, ll[0]*u), xj[j0], xj[j0+2])[0]
        b[j0+1] += quad(sp.lambdify(x, ll[1]*u), xj[j0], xj[j0+2])[0]
        b[j0+2] += quad(sp.lambdify(x, ll[2]*u), xj[j0], xj[j0+2])[0]
    return A, b


In [ ]:
N = 2*10
u = 10*(x-1)**2-1
xj = np.linspace(1, 2, N+1)
A, b = assemble_fem2(u, xj)
uh2 = np.linalg.inv(A) @ b
plt.figure(figsize=(5, 3))
yj = np.linspace(1, 2, 100)
plt.plot(xj, uh2, 'b', yj, 10*(yj-1)**2-1, 'r--')
plt.legend(['FEM', 'Exact']);

Lets try the more difficult function 

$$
u(x) = \exp(\cos x), \quad x \in [-1, 1].
$$

that required 20 Legendre or Chebyshev coefficients for full machine precision. Create first a function that assembles the problem for $N$ elements with linear polynomials

In [ ]:
def assemble_fem(N, u, xj):
    l0 = lambda j: (x-xj[j+1])/(xj[j]-xj[j+1])
    l1 = lambda j: (x-xj[j])/(xj[j+1]-xj[j])
    A = np.zeros((N+1, N+1))
    b = np.zeros(N+1)
    for i in range(N):
        A[i, i] += quad(sp.lambdify(x, l0(i)**2), xj[i], xj[i+1])[0]
        A[i+1, i+1] += quad(sp.lambdify(x, l1(i)**2), xj[i], xj[i+1])[0]
        aij = quad(sp.lambdify(x, l0(i)*l1(i)), xj[i], xj[i+1])[0]
        A[i, i+1] += aij
        A[i+1, i] += aij    
        b[i] += quad(sp.lambdify(x, l0(i)*u), xj[i], xj[i+1])[0]
        b[i+1] += quad(sp.lambdify(x, l1(i)*u), xj[i], xj[i+1])[0]
    return A, b

Now loop over a range of mesh sizes and assemble both the problem using linear basis functions `assemble_fem` and the problem using second order polynomials `assemble_fem2`. When finished compute the $L^2$ error norm using only the node values

In [ ]:
def L2_error(uh, ue, xj):
    uej = ue(xj)
    return np.sqrt(np.trapz((uh-uej)**2, dx=xj[1]-xj[0]))
    
u = sp.exp(sp.cos(x))
ue = sp.lambdify(x, u) 
err = []
err2 = []
for n in range(2, 30, 4):
    N = 2*n
    xj = np.linspace(-1, 1, N+1)
    A, b = assemble_fem(N, u, xj)
    uh = np.linalg.inv(A) @ b
    A2, b2 = assemble_fem2(u, xj)
    uh2 = np.linalg.inv(A2) @ b2
    err.append(L2_error(uh, ue, xj))
    err2.append(L2_error(uh2, ue, xj))

The FEM approximation using linear polynomials should be second order accurate, because the linear $\ell_0$ and $\ell_1$ have Taylor expansions where the first neglected term is proportional to $\Delta x^2$. Similarly, the FEM approximation using second order polynomials should be fourth order. As stated in [lecture 9](https://matmek-4270.github.io/matmek4270-book/lecture9.html) the Chebyshev and Legendre approximations have spectral accuracy, which is faster than any finite order. This is easy to illustrate. In the figure below we plot the $L^2$ errors of the Legendre/Chebyshev approximations as well as the FEM. Note how the spectral methods accellerate towards machine precision, whereas the error for FEM disappears at a rate proportional to $N^{-2}$ and $N^{-4}$. 

In [ ]:
err_cl = np.load('./err_u.npy')
plt.figure(figsize=(5, 3))
plt.loglog(np.arange(0, 25, 2), err_cl[0], '+',
           np.arange(0, 25, 2), err_cl[1], 'ko',
           np.arange(2, 30, 4)*2, err, 'g', 
           np.arange(2, 30, 4)*2, err2, 'y', 
           fillstyle='none')
from plotslopes import slope_marker
slope_marker((12, err[1]), (2, -1))
slope_marker((20, err2[2]), (4, -1))
plt.legend(['Chebyshev', 'Legendre', 'FEM']);

The high order of accuracy is 

The finite element functions are also a bit tricky to evaluate outside the mesh points. How do you copute 

$$
u(x) = \sum_{j=0}^N \hat{u}_j \psi_j(x)
$$

for any given $x \in \Omega$ if $x$ is not a mesh point?

Here you need to find out which element $x$ is in, and then you need to evaluate using all the nonzero basis functions in that element. If there are $d$ nonzero basis functions in each element, then you need to evaluate

$$
u(x) = \sum_{i=0}^{d-1} \hat{u}_{2e+i} \psi_{2e+i}(x)
$$

where $e$ is the element number that $x$ belongs to.

In [ ]:
from lagrange import Lagrangefunction
def fem_eval(uh, y, xj, d=1):
    h = xj[d]-xj[0]
    elem = int(np.floor((y-xj[0])/h))
    ll = Lagrangebasis(xj[d*elem:d*(elem+1)+1])
    print(elem)
    print(ll)
    f = Lagrangefunction(uh[d*elem:d*(elem+1)+1], ll)
    return f.subs(x, y)
    

In [ ]:
N = 20
xj = np.linspace(-1, 1, N+1)
A, b = assemble_fem2(u, xj)
uh = np.linalg.inv(A) @ b
fem_eval(uh, -0.95, xj, d=2), u.subs(x, -0.95)

In [ ]:
uh